<a href="https://colab.research.google.com/github/TrifanLucian/Performance-Analysis-of-Deep-Learning-Algorithms/blob/main/FlickAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import urllib.request
from flickrapi import FlickrAPI
import time
import urllib.error
from PIL import Image

# Flickr API credentials
FLICKR_PUBLIC = 'b772eeebe6f9c823f99b6b51fae41fe7'
FLICKR_SECRET = 'b97d32c828e6a8a2'

# Initialize FlickrAPI object
flickr = FlickrAPI(FLICKR_PUBLIC, FLICKR_SECRET, format='parsed-json')

# Define your search parameters
search_params = {
    'text': 'apples',
    'per_page': 500,  # Increase per_page to 500
    'media': 'photos',  # Search for photos
    'sort': 'relevance',  # Sort by relevance
    'safe_search': 1,  # Safe search mode on
    'content_type': 1,  # Only photos
    'extras': 'url_o',  # Get the original image URL
}

# Directory to save the downloaded images
output_directory = r"C:\Users\Lucian\Downloads\fructe\mere_resized"

# Ensure output directory exists
os.makedirs(output_directory, exist_ok=True)

# Download counter
downloaded_count = 0

# Perform searches until 3000 photos are downloaded
while downloaded_count < 10000:
    try:
        # Perform the search
        response = flickr.photos.search(**search_params)

        # Parse the response and download the photos
        if response['stat'] == 'ok':
            photos = response['photos']['photo']
            for photo in photos:
                if 'url_o' in photo:
                    photo_url = photo['url_o']
                    # Extract the filename from the photo URL
                    filename = os.path.basename(photo_url)
                    # Check if the filename already exists in the output directory
                    if not os.path.exists(os.path.join(output_directory, filename)):
                        # Download the photo
                        output_path = os.path.join(output_directory, filename)
                        urllib.request.urlretrieve(photo_url, output_path)
                        print(f"Downloaded: {filename}")

                        # Resize the downloaded image to 256x256 pixels
                        try:
                            with Image.open(output_path) as img:
                                resized_img = img.resize((256, 256))
                                resized_img.save(output_path)
                                print(f"Resized: {filename} to 256x256")
                        except Exception as e:
                            print(f"Error resizing image {filename}: {e}")

                        downloaded_count += 1
                        if downloaded_count >= 10000:
                            break  # Stop if 3000 photos are downloaded
                    else:
                        print(f"Skipping {filename}: File already exists in output directory.")
        else:
            print("Failed to perform Flickr API search.")
            break
    except (urllib.error.URLError, http.client.RemoteDisconnected) as e:
        print(f"Error: {e}. Retrying in 5 seconds...")
        time.sleep(5)  # Retry after 5 seconds

# Inform if the target number of photos is reached
print(f"Downloaded and resized {downloaded_count} photos.")